# TSA Chapter 4: Case Study - Model Comparison

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch4/TSA_ch4_case_model_comparison/TSA_ch4_case_model_comparison.ipynb)

Comparing SARIMA specifications using AIC/BIC.

In [ ]:
!pip install numpy pandas matplotlib statsmodels -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Load airline passengers
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv',
                    parse_dates=['Month'], index_col='Month')
passengers = data['Passengers']
passengers.index.freq = 'MS'
log_passengers = np.log(passengers)

In [ ]:
specs = [((0,1,1),(0,1,1,12)), ((1,1,0),(1,1,0,12)), ((1,1,1),(0,1,1,12)), ((1,1,1),(1,1,1,12))]
results = []
for o, so in specs:
    try:
        m = SARIMAX(log_passengers, order=o, seasonal_order=so, enforce_stationarity=False, enforce_invertibility=False)
        r = m.fit(disp=False); results.append({'label': f'({o[0]},{o[1]},{o[2]})({so[0]},{so[1]},{so[2]})', 'aic': r.aic, 'bic': r.bic})
    except: pass
results.sort(key=lambda x: x['aic'])
fig, ax = plt.subplots(figsize=(10, 4))
y = np.arange(len(results)); w = 0.35
ax.barh(y-w/2, [r['aic'] for r in results], w, color=BLUE, alpha=0.8, label='AIC')
ax.barh(y+w/2, [r['bic'] for r in results], w, color=ORANGE, alpha=0.6, label='BIC')
ax.set_yticks(y); ax.set_yticklabels([r['label'] for r in results]); ax.invert_yaxis()
ax.set_title('SARIMA Model Comparison', fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
plt.tight_layout(); save_chart(fig, 'ch4_case_model_comparison'); plt.show()